# The L-shape
https://www.sintef.no/globalassets/project/evitameeting/2005/lcurve.pdf

## Packages

In [ ]:
using SparseArrays
using LinearAlgebra
using DIVAnd
using Makie, CairoMakie
using Statistics
using JupyterFormatter
# enable_autoformat()

## Data

In [ ]:
ND = 100
NX = 500

len = 0.1
# obs. error variance normalized by the background error variance
epsilon2 = 1.0

fun(x) = 2 * sin.(10 .* x) .+ (0 .- 1) .* x
x = 0.5 .+ 0.25 .* randn(ND);
f = fun.(x) + 0.1 * randn(ND)
# final grid
xi = collect(range(0, stop = 1, length = NX));

@show size(xi)

# all points are valid points
mask = trues(size(xi));

fi = fun.(xi)

pm = ones(size(xi)) / (xi[2] - xi[1]);

Ntest = 100
ee1 = zeros(Float64, Ntest)
ee2 = zeros(Float64, Ntest)
ee3 = zeros(Float64, Ntest)
epsval = zeros(Float64, Ntest);

In [ ]:
?DIVAnd_norms

In [ ]:
for jjj = 1:Ntest
    epsilon2 = 10^(-8.0 + jjj / Ntest * 10)
    fi, s = DIVAndrun(mask, (pm,), (xi,), (x,), f, (len,), epsilon2)

    #pcolor(xi[:,10,:],zi[:,10,:],fi[:,10,:],shading="nearest"),colorbar()

    myz = 2
    #pcolor(xi[:,:,myz],yi[:,:,myz],fi[:,:,myz],shading="nearest"),colorbar()

    ee1[jjj], ee2[jjj], ee3[jjj], epsval[jjj] = DIVAnd_norms(fi, s)
end

### Plot for a given value

In [ ]:
mychoice = 70
@show log10.(ee2[mychoice] .* (epsval[mychoice])), log10.(ee1[mychoice]), epsval[mychoice]
fig = Figure()
ax = Axis(
    fig[1, 1],
    title = "L-shape, \$ \\log_{10} \\epsilon^2 \$ (error variance on observations) by color",
    xlabel = "\$\\log_{10} ||H x -yo||^2\$",
    ylabel = "\$\\log_{10} ||x||^2\$",
)
sc = scatter!(ax, log10.(ee2 .* (epsval)), log10.(ee1), color = log10.(epsval))
scatter!(
    ax,
    log10.(ee2[mychoice] .* (epsval[mychoice])),
    log10.(ee1[mychoice]),
    color = :red,
    markersize = 12,
)
Colorbar(fig[1, 2], sc)
fig

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1], xlabel = "\$\\log_{10} \\epsilon^2\$")
lines!(ax, log10.(epsval), log10.(ee2 .* epsval))
lines!(ax, log10.(epsval), log10.(ee1))
scatter!(ax, log10.(epsval[mychoice]), log10.(ee2[mychoice] .* (epsval[mychoice])))
scatter!(ax, log10.(epsval[mychoice]), log10.(ee1[mychoice]))
fig

In [ ]:
epsilon2 = epsval[mychoice]
fi, s = DIVAndrun(mask, (pm,), (xi,), (x,), f, (len,), epsilon2);

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1], xlabel = "\$\\log_{10} \\epsilon^2\$")
lines!(ax, xi, fi, label = "Interpolated field")
lines!(ax, xi, fun.(xi), label = "True field")
scatter!(ax, x, f, label = "Observations")
axislegend()
fig